## Data clean-up

In [5]:
import matplotlib.pyplot as plt
import pandas as pd
import os 

plt.ioff()

csfont = {'fontname':'AnjaliOldLipi', 'fontsize':30}
hfont = {'fontname':'AnjaliOldLipi', 'fontsize':30}

In [119]:
import wandb

api = wandb.Api()

def get_runs_filters(api, group="sfols", env_name = "Delivery-v0", timestep_col = "learning_timestep"):

    runs = api.runs("davidguillermo/sfcomp")
    
    summary_list, config_list, name_list = [], [], []

    df = pd.DataFrame(columns=["timestep"])
    
    for run in runs: 
        if run.group == group and run.config["env"]["gym_name"] == env_name:

            data = run.history(x_axis=timestep_col, keys=["learning/fsa_reward"])

            df.loc[:, "timestep"] = data[timestep_col]
            df.loc[:, run.name] = data["learning/fsa_reward"]
    
        # # .config contains the hyperparameters.
        # #  We remove special values that start with _.
        # config_list.append(
        #     {k: v for k,v in run.config.items()
        #       if not k.startswith('_')})
    
        # # .name is the human-readable name of the run.
        # name_list.append(run.name)

    return df

def drop_columns(df):
    todrop = []
    for col in df.columns:
        if '__MIN' in col or '__MAX' in col or (col.endswith("_step") and col !="global_step"):
            todrop.append(col) 
    todrop

    return df.drop(columns=todrop)


def cleandf(df):
    
    df = drop_columns(df)
    #rename 

    print(df.columns)

    renames = {}

    newdf = pd.DataFrame(columns=["step", "mean", "std"])

    newdf.loc[:, "mean"] = df.loc[ : , df.columns!='timestep'].mean(axis=1)
    newdf.loc[:, "std"] = df.loc[ : , df.columns!='timestep'].std(axis=1)
    newdf.loc[:, "step"] = df.timestep

    return newdf

In [107]:
delivery_sfols = cleandf(sfols_delivery)

sr, timemark = len(delivery_sfols), delivery_sfols.step.max()

mean, std = delivery_sfols.iloc[-1, [1,2]]

for i, r in enumerate(range(timemark+500, 160001, 500)):

    delivery_sfols.loc[sr + i + 1, :] = [r, mean, std]

Index(['timestep', 'lively-terrain-597', 'absurd-snow-596',
       'honest-cherry-595', 'autumn-wind-594', 'generous-sunset-593',
       'driven-sound-591', 'grateful-forest-590', 'vivid-sky-589',
       'bright-bee-587', 'magic-shape-586', 'frosty-hill-585',
       'usual-elevator-584', 'charmed-terrain-583', 'proud-dew-581',
       'clear-jazz-580', 'chocolate-meadow-579', 'major-snowball-578',
       'feasible-gorge-577', 'expert-feather-575', 'lemon-wood-574'],
      dtype='object')


In [108]:
delivery_sfols

,step,mean,std
0,500,-1001.0,1.297771
1,1000,-1001.3,1.894591
2,1500,-1000.6,0.680557
3,2000,-1001.6,1.788854
4,2500,-1000.7,1.301821
...,...,...,...
316,158000,-50.0,12.31174
317,158500,-50.0,12.31174
318,159000,-50.0,12.31174
319,159500,-50.0,12.31174


### Delivery

In [112]:
# sfols_delivery = get_runs_filters(api)
# delivery_sfols = cleandf(sfols_delivery)

# lof_delivery = get_runs_filters(api, group="lof", env_name = "Delivery-v0", timestep_col = "global_step")
# delivery_lof = cleandf(lof_delivery)

In [113]:
# delivery_sfols.to_csv("SFOLS-Delivery.csv", index=False)
# delivery_lof.to_csv("LOF-Delivery.csv", index=False)

### Office

In [121]:
lof_office = get_runs_filters(api, group="lof", env_name = "Office-v0", timestep_col = "global_step")
office_lof = cleandf(lof_office)
office_lof
office_lof.to_csv("LOF-Office.csv", index=False)

Index(['timestep', 'polished-salad-491', 'neat-morning-490', 'mild-grass-488',
       'dauntless-moon-487', 'cosmic-frost-485', 'exalted-pond-484',
       'true-yogurt-483', 'wild-planet-481', 'rural-music-480',
       'autumn-cloud-478', 'rural-meadow-477', 'splendid-universe-475',
       'stellar-energy-474', 'toasty-sky-473', 'whole-forest-471',
       'rich-terrain-470', 'faithful-armadillo-469', 'whole-sun-466',
       'celestial-resonance-465', 'solar-snowflake-462', 'woven-planet-332',
       'whole-smoke-331', 'fearless-puddle-330', 'peachy-music-329',
       'ethereal-sponge-328', 'zany-smoke-327', 'sandy-meadow-325',
       'valiant-cosmos-322', 'clear-mountain-320', 'giddy-surf-317',
       'lyric-flower-281', 'bumbling-dream-280', 'brisk-monkey-279',
       'dulcet-disco-278', 'ethereal-salad-277', 'eternal-sun-274',
       'youthful-darkness-268', 'whole-forest-262', 'jolly-paper-260',
       'lively-deluge-256'],
      dtype='object')
